[1] https://eprint.iacr.org/2022/057.pdf

[2] https://link.springer.com/content/pdf/10.1007/11761679_17.pdf

In [1]:
from tqdm import tqdm, trange
from sign import *

from secure_code.ntrugen import adj
from secure_code.fft import div_fft, sub_fft

import matplotlib.pyplot as plt
import numpy as np

In [2]:
def gram_approximation(vectors):
    '''Given random vectors v chosen uniformly on the parallelepiped P(V), approximates the gram matrix G = V^t V.
    This is the first step of Algorithm 1 in [2], given by Lemma 1 (Gram Leakage):
            V^t V = 3 Exp[v^t v]
    '''
    n = vectors.shape[1]
    A = np.zeros((n, n), 'int64')
    N = len(vectors)
    for v in tqdm(vectors, desc = 'Gram Approximation'):
        A += np.mat(v).T @ np.mat(v)
    return 3*A/N

In [3]:
def hypercube_transformation(G, vectors):
    '''Given (an approximation of) the gram matrix G of V, computes the map L such that P(C) is an hypercube, where C = VL.
    This is step 2 and 3 of Algorithm 1 in [2], given by Lemma 2 (Hypercube Transformation).'''
    L = np.linalg.cholesky(np.linalg.inv(G))
    M = vectors @ L # computes all the vL at once 
    return L, M

In [4]:
def fourth_moment(w, vectors):
    '''mom{V,k}(w) = Exp[(u, w)^k].'''
    return np.mean((vectors @ w)^4)

In [5]:
def fourth_moment_gradient(w, vectors):
    '''∇mom{V,4}(w) = Exp[∇((u, w)⁴)] = 4 Exp[(u, w)³u].'''
    coeffs = (vectors @ w)^3
    return 4*(coeffs @ vectors)/len(vectors)

In [6]:
def learn_hidden_hypercube(n, δ, vectors):
    '''Algorithm 2 (Solving the Hidden Hypercube Problem by Gradient Descent) in [2].'''
    w = np.random.random(n)
    w /= np.linalg.norm(w)

    g = fourth_moment_gradient(w, vectors)
    wnew = w - δ*g
    wnew /= np.linalg.norm(wnew)

    while fourth_moment(wnew, vectors) < fourth_moment(w, vectors):
        w = wnew
        g = fourth_moment_gradient(w, vectors)
        wnew = w - δ*g
        wnew /= np.linalg.norm(wnew)

    return w

In [7]:
def poly2mat(poly):
    mat = []
    poly = list(poly)
    for _ in range(128):
        mat.append(poly)
        poly = [-poly[-1]] + poly[:-1] # x*poly % x^n + 1
    return matrix(mat)

In [8]:
#sk = np.load("data/my_sk.npy")
pk = np.load("data/pubkey.npy")
sigs = np.load("data/sigs.npy", allow_pickle = True)

In [9]:
signatures = sigs
message = b'fcsc2022'

In [10]:
vectors = []
n = len(signatures[0][1][0])
signature_bound = 2*n*q

for signature in tqdm(signatures, desc = 'Processing Signatures'):
    _, (s0, s1) = signature
    s = list(s0) + list(s1)
    if np.inner(s, s) < 2*n*q:
        vectors.append(s)

print(f'Found {len(vectors)} valid among {len(signatures)} signatures')
vectors = np.array(vectors)

Processing Signatures: 100%|██████████| 300000/300000 [00:33<00:00, 8875.41it/s] 


Found 299944 valid among 300000 signatures


In [11]:
#G = gram_approximation(vectors)
G = np.load('G.npy', allow_pickle=True)
np.save('G.npy', G, allow_pickle=True)

Gram Approximation: 100%|██████████| 299944/299944 [00:52<00:00, 5675.77it/s]


In [12]:
L, cube_vectors = hypercube_transformation(4*G, vectors)

In [13]:
def solve(L, δ, cube_vectors):
    w = learn_hidden_hypercube(len(vectors[0]), δ, cube_vectors)
    wl = w @ np.linalg.inv(L)
    return wl

In [14]:
candidates = []

In [46]:
best_candidates = []

In [47]:
δ = 0.7 # according to [2] this works well experimentally

while 1:
    print('iter...')
    wl = solve(L, δ, cube_vectors)
    wr = np.round(wl)
    g =  wr[:128]
    f = -wr[128:]

    g = [int(x)%q for x in g]
    f = [int(x)%q for x in f]

    if [int(y)%q for y in mul_zq(f, pk)] == g:
        print('='*30, 'CANDIDATE', '='*30)
        print('wl =', wl)
        print('wr =', wr)
        print('Norm:', gs_norm(f, g, q))
        candidates.append(wl)
        try:
            F, G = ntru_solve(f, g)
            print('[+] Solved NTRU')
            sk = f, g, F, G
            print('sk =', sk)
        except ValueError:
            print('[!] Could not solve NTRU')
            continue

        best_candidates.append(wl)
        sgn = sign(sk, message)
        if verify(pk, message, sgn):
            print('[+] Successfully verified signature!')            

iter...
============================== CANDIDATE ==============================
wl = [-4.31751165e+00  9.31160817e-01  1.94702198e+00 -4.28532319e-02
 -6.85037269e+00 -1.09458837e+01 -8.53658315e-01  1.84100400e+00
 -1.99329184e+00 -6.92037915e+00  3.15318209e+00 -9.61244259e-01
  4.97486305e+00  4.93063759e+00  6.87902016e+00  3.79288839e-02
  3.87961678e+00 -6.89563993e+00  3.75476576e+00  1.95463910e+00
 -9.21852770e-01  1.18868659e+01 -6.08514827e+00 -9.59151319e-01
  3.20634381e+00 -4.84612373e+00 -8.63774209e-02 -3.33678930e-01
  1.40178920e+01  4.13417088e+00  2.03726616e+00 -3.96912358e+00
  2.06150120e+00  2.80707212e+00 -1.79527029e+00 -9.00525271e-02
 -6.01657415e+00  4.83824221e-03  4.08859132e+00 -5.08249252e+00
 -5.78509432e+00 -8.91081992e+00 -5.00547888e+00 -6.11291029e+00
  8.91978512e+00 -1.82184593e+00 -7.04764410e-01 -3.92639582e+00
 -2.94994739e+00  5.74426496e+00  7.08150388e+00  1.69627747e+00
 -7.13668030e+00 -1.19775865e+01  3.95599223e+00  2.16968777e+00
  3.1

iter...
iter...
iter...
============================== CANDIDATE ==============================
wl = [ 4.98400606e+00  7.06490772e+00  4.29638613e-02  4.26243548e+00
 -7.28092184e+00  4.06910264e+00  2.04122772e+00 -9.92339280e-01
  1.20643691e+01 -5.95499917e+00 -1.22910475e+00  2.84756808e+00
 -4.68351798e+00 -1.16834962e-01  7.57595259e-02  1.38309133e+01
  3.72685131e+00  1.77694231e+00 -3.85216552e+00  2.07399600e+00
  3.20958185e+00 -2.09311464e+00 -2.37890756e-01 -6.07332993e+00
  1.17073250e-02  4.02760047e+00 -4.93747352e+00 -6.08392772e+00
 -8.87086693e+00 -5.06025067e+00 -5.87714429e+00  9.06769302e+00
 -1.96984804e+00 -7.47777990e-01 -3.97756005e+00 -3.08216508e+00
  6.09612089e+00  6.97097915e+00  2.23291954e+00 -6.93650533e+00
 -1.21193778e+01  3.96561023e+00  1.96858330e+00  2.94504786e+00
 -2.72166180e+00 -1.31479764e+01 -1.19135570e-01  9.79191158e+00
  4.93327519e+00  4.24018360e+00  1.01671906e+00 -1.30666127e+01
  6.20294730e-01  2.93633732e+00  3.23934745e-01  1.09

============================== CANDIDATE ==============================
wl = [ 9.66763732e-01  2.96159659e+00 -6.92083069e-03  1.10565102e+01
  2.99917491e+00 -5.01875509e+00  9.87717044e+00 -3.11618315e+00
  1.09826581e+01 -1.91216289e+00  5.81106968e-02  9.06488473e-01
 -3.06365483e+00 -4.07101642e+00  4.01146190e+00  1.03143294e+01
  9.79609506e+00 -9.24677360e+00 -1.23553784e+01  9.91853322e+00
  3.29536097e+00  1.20714751e+01 -3.05046067e+00 -1.31429522e+01
 -1.29139488e+00  2.91977280e+00  1.45205779e-01 -3.47368632e-02
 -2.87417494e+00 -9.12268169e+00  1.20048829e+01  3.10582674e+00
  8.88636190e+00 -4.27403611e+00  1.04112401e-01 -5.23168504e+00
  1.10310271e+01  8.05506638e+00  3.91632504e+00  1.83914304e+00
 -2.98904920e+00 -1.09757185e+01 -1.23775217e+00 -8.10651272e-01
  6.04962668e+00  1.01981737e+01  1.16127483e+00 -9.00274269e-01
  2.86324177e+00 -4.78031068e+00  1.59079448e-01  6.13292192e+00
 -2.80262925e+00 -1.89578686e-01 -1.96612489e-01  2.26116064e+00
 -4.98923513e

[+] Solved NTRU
sk = ([5, 767, 0, 768, 0, 2, 0, 0, 2, 0, 0, 767, 3, 768, 765, 0, 767, 0, 2, 768, 768, 0, 3, 0, 1, 2, 767, 0, 1, 768, 0, 0, 1, 768, 1, 1, 0, 2, 1, 0, 1, 0, 2, 1, 0, 766, 768, 768, 0, 766, 767, 768, 767, 0, 0, 2, 0, 0, 767, 768, 3, 1, 768, 0, 3, 4, 0, 0, 2, 766, 768, 0, 768, 764, 0, 2, 2, 1, 1, 0, 768, 4, 0, 0, 0, 0, 1, 1, 768, 1, 1, 765, 0, 0, 2, 0, 767, 0, 768, 0, 767, 767, 0, 1, 0, 1, 0, 0, 765, 0, 768, 0, 0, 0, 766, 768, 768, 0, 768, 768, 1, 1, 768, 1, 768, 0, 3, 765], [0, 2, 2, 0, 1, 0, 767, 1, 4, 0, 3, 0, 0, 1, 0, 768, 2, 1, 765, 1, 766, 4, 0, 767, 767, 0, 768, 0, 767, 768, 2, 767, 0, 0, 768, 0, 0, 767, 3, 768, 766, 768, 2, 1, 4, 0, 765, 0, 0, 1, 4, 1, 768, 0, 768, 0, 3, 768, 1, 767, 765, 0, 768, 0, 768, 768, 2, 767, 2, 768, 0, 767, 0, 0, 5, 765, 2, 1, 764, 1, 2, 2, 0, 0, 0, 1, 0, 0, 765, 0, 768, 0, 0, 0, 767, 766, 0, 0, 1, 0, 1, 0, 0, 4, 768, 768, 3, 0, 0, 764, 766, 0, 0, 0, 767, 2, 3, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 1], [-2908, -286, -157, -1647, -1734, 2398, -201, 

KeyboardInterrupt: 

In [48]:
len(candidates)

10

In [50]:
backup = np.load('candidates.npy')

In [53]:
np.save('candidates.npy', candidates)

In [54]:
wrs = np.rint(candidates)
for i, wr in enumerate(wrs):
    g = wr[:128]
    f = -wr[128:]
    assert vector([int(x) for x in sub(mul_zq(f, pk), g)]) % q == 0, i

In [55]:
#A = poly2mat(list(pk)).stack(identity_matrix(128)*q).augment(-identity_matrix(128).stack(zero_matrix(128)))
A = matrix([[int(x) for x in wr] for wr in wrs])
B = A.LLL()

In [56]:
for wr in B:
    g = wr[:128]
    f = -wr[128:]
    try:
        F, G = ntru_solve(f, g)
    except:
        continue
    print('yaaay')
    break

yaaay


In [59]:
F, G = ntru_solve(f, g)

In [80]:
sk = f, g, F, G
sgn = sign(sk, message)
verify(pk, message, sgn)

True

In [68]:
from minipwn import remote
io = remote('challenges.france-cybersecurity-challenge.fr', 2102)

In [71]:
io.recvline()

b'Enter the signature:\n'

In [72]:
message = b'spaceship_42bcc12607afa08c'
r, s = sign(sk, message)
io.sendline(r.hex())
io.sendline(str(s))

In [74]:
io.recvline()

b'FCSC{29ae532065c3cf7f860a8978fb4ffa6354f1a3e10ffb92abcafdc59cdcb14456}\n'